In [116]:
import os
import time
import glob

In [117]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [118]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics.classification import AveragePrecision

In [119]:
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, average_precision_score

In [120]:
class Config:
    KAGGLE = False
    ROOT = '../'
    if KAGGLE:
        ROOT = '/kaggle/input/'
    DATA_DIR = f'{ROOT}tlvmc-parkinsons-freezing-gait-prediction/'
    TRAIN_DIR = f'{ROOT}tlvmc-parkinsons-freezing-gait-prediction/train/'
    TDCSFOG_DIR = f'{ROOT}tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/'
    DEFOG_DIR = f'{ROOT}tlvmc-parkinsons-freezing-gait-prediction/train/defog/'
    CHECKPOINT_PATH = f'{ROOT}/checkpoints/'

    batch_size = 1024
    window_size = 30*128
    window_future = 8*128
    window_past = window_size - window_future

    model_dropout = 0.2
    model_hidden = 128
    model_nblocks = 2

    lr = 0.001
    num_epochs = 8
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    feature_list = ['AccV', 'AccML', 'AccAP']
    label_list = ['StartHesitation', 'Turn', 'Walking']

cfg = Config()

In [121]:
cfg.device

'cpu'

## Data - Preprocessing

In [122]:
class FOGDataset(Dataset):
    def __init__(self, fpaths, scale=9.806, test=False):
        super(FOGDataset, self).__init__()
        tm = time.time()
        self.test = test
        self.fpaths = fpaths
        self.f_ids = [os.path.basename(f)[:-4] for f in self.fpaths]
        self.curr_df_idx = 0
        self.curr_row_idx = 0
        self.dfs = [np.array(pd.read_csv(f)) for f in fpaths]
        self.end_indices = []
        self.scale = scale
        
        self.length = 0
        for df in self.dfs:
            self.length += df.shape[0]
            self.end_indices.append(self.length)
            
        print(f"Dataset initialized in {time.time() - tm} secs!")
        
    def pad(self, df, time_start):
        if df.shape[0] == cfg.window_size:
            return df
        
        npad = cfg.window_size - df.shape[0]
        padzeros = np.zeros((npad, 3))
        if time_start <= 0:
            df = np.concatenate((padzeros, df), axis=0)
        else:
            df = np.concatenate((df, padzeros), axis=0)
        return df
            
    def __getitem__(self, index):
        for i,e in enumerate(self.end_indices):
            if index >= e:
                continue
            df_idx = i
            break
            
        curr_df = self.dfs[i]
        row_idx = curr_df.shape[0] - (self.end_indices[i] - index)
        _id = self.f_ids[i] + "_" + str(row_idx)
        
        x = self.pad(curr_df[row_idx-cfg.window_past:row_idx+cfg.window_future, 1:4], row_idx-cfg.window_past )
        x = torch.tensor(x)/self.scale
        
        if self.test == True:
            return _id, x
        
        y = curr_df[row_idx, -3:].astype('float')
        y = torch.tensor(y)
        
        return x, y
    
    def __len__(self):
        return self.length

In [123]:
# Analysis of positive instances in each fold of our CV folds

SH = []
T = []
W = []

# Here I am using the metadata file available during training. Since the code will run again during submission, if 
# I used the usual file from the competition folder, it would have been updated with the test files too.
metadata = pd.read_csv("../tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv")

for f in tqdm(metadata['Id']):
    fpath = f"../tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/{f}.csv"
    df = pd.read_csv(fpath)
    
    SH.append(np.sum(df['StartHesitation']))
    T.append(np.sum(df['Turn']))
    W.append(np.sum(df['Walking']))
    
print(f"32 files have positive values in all 3 classes")

metadata['SH'] = SH
metadata['T'] = T
metadata['W'] = W

sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, valid_index) in enumerate(sgkf.split(X=metadata['Id'], y=[1]*len(metadata), groups=metadata['Subject'])):
    print(f"Fold = {i}")
    train_ids = metadata.loc[train_index, 'Id']
    valid_ids = metadata.loc[valid_index, 'Id']
    
    print(f"Length of Train = {len(train_ids)}, Length of Valid = {len(valid_ids)}")
    n1_sum = metadata.loc[train_index, 'SH'].sum()
    n2_sum = metadata.loc[train_index, 'T'].sum()
    n3_sum = metadata.loc[train_index, 'W'].sum()
    print(f"Train classes: {n1_sum:,}, {n2_sum:,}, {n3_sum:,}")
    
    n1_sum = metadata.loc[valid_index, 'SH'].sum()
    n2_sum = metadata.loc[valid_index, 'T'].sum()
    n3_sum = metadata.loc[valid_index, 'W'].sum()
    print(f"Valid classes: {n1_sum:,}, {n2_sum:,}, {n3_sum:,}")
    
# # FOLD 2 is the most well balanced

  0%|          | 0/833 [00:00<?, ?it/s]

32 files have positive values in all 3 classes
Fold = 0
Length of Train = 672, Length of Valid = 161
Train classes: 287,832, 1,462,652, 175,633
Valid classes: 16,958, 216,130, 32,205
Fold = 1
Length of Train = 613, Length of Valid = 220
Train classes: 51,748, 909,505, 65,242
Valid classes: 253,042, 769,277, 142,596
Fold = 2
Length of Train = 703, Length of Valid = 130
Train classes: 271,881, 1,332,746, 183,673
Valid classes: 32,909, 346,036, 24,165
Fold = 3
Length of Train = 649, Length of Valid = 184
Train classes: 303,710, 1,517,147, 205,196
Valid classes: 1,080, 161,635, 2,642
Fold = 4
Length of Train = 695, Length of Valid = 138
Train classes: 303,989, 1,493,078, 201,608
Valid classes: 801, 185,704, 6,230


In [124]:
# The actual train-test split (based on Fold 2)

metadata = pd.read_csv(cfg.DATA_DIR + "tdcsfog_metadata.csv")
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, valid_index) in enumerate(sgkf.split(X=metadata['Id'], y=[1]*len(metadata), groups=metadata['Subject'])):
    if i != 2:
        continue
    print(f"Fold = {i}")
    train_ids = metadata.loc[train_index, 'Id']
    valid_ids = metadata.loc[valid_index, 'Id']
    print(f"Length of Train = {len(train_ids)}, Length of Valid = {len(valid_ids)}")
    
    if i == 2:
        break
        
train_fpaths = [f"{cfg.DATA_DIR}train/tdcsfog/{_id}.csv" for _id in train_ids]
valid_fpaths = [f"{cfg.DATA_DIR}train/tdcsfog/{_id}.csv" for _id in valid_ids]

Fold = 2
Length of Train = 703, Length of Valid = 130


In [125]:
#import glob module 
import glob
# get all the file paths in to list for the test directory
test_fpaths = glob.glob(f"{cfg.DATA_DIR}test/tdcsfog/*.csv")
test_fpaths

['../tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/003f117e14.csv']

# TODO: Generalize preprocessing

In [126]:
tdcsfog_train = FOGDataset(train_fpaths)
tdcsfog_train_loader = DataLoader(tdcsfog_train, batch_size=cfg.batch_size, shuffle=True)

Dataset initialized in 7.881666898727417 secs!


In [127]:
tdcsfog_valid = FOGDataset(valid_fpaths)
tdcsfog_valid_loader = DataLoader(tdcsfog_valid, batch_size=cfg.batch_size)

Dataset initialized in 1.47963285446167 secs!


In [128]:
tdcsfog_test = FOGDataset(test_fpaths)
tdcsfog_test_loader = DataLoader(tdcsfog_test, batch_size=cfg.batch_size)

Dataset initialized in 0.009447813034057617 secs!


## Model

In [129]:
def _block(in_features, out_features, drop_rate):
    return nn.Sequential(
        nn.Linear(in_features, out_features),
        nn.BatchNorm1d(out_features),
        nn.ReLU(),
        nn.Dropout(drop_rate)
    )

class FOGModel(nn.Module):
    def __init__(self, p=cfg.model_dropout, dim=cfg.model_hidden, nblocks=cfg.model_nblocks):
        super(FOGModel, self).__init__()
        #self.dropout = nn.Dropout(p)
        self.in_layer = nn.Linear(cfg.window_size*3, dim)
        self.blocks = nn.Sequential(*[_block(dim, dim, p) for _ in range(nblocks)])
        self.out_layer = nn.Linear(dim, 3)
        
    def forward(self, x):
        x = x.view(-1, cfg.window_size*3)
        x = self.in_layer(x)
        for block in self.blocks:
            x = block(x)
        x = self.out_layer(x)
        return x

class FOGTransformer(nn.Module):
    def __init__(self, p=cfg.model_dropout, dim=cfg.model_hidden, nblocks=cfg.model_nblocks):
        super(FOGTransformer, self).__init__()
        self.dropout = nn.Dropout(p)
        self.in_layer = nn.Linear(cfg.window_size*3, dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=8, dim_feedforward=dim)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=1, mask_check=False)

        self.out_layer = nn.Linear(dim, 3)

    def forward(self, x):
        x = x.view(-1, cfg.window_size*3)
        x = self.in_layer(x)
        x = self.transformer(x)
        x = self.out_layer(x)
        return x

In [130]:
# get the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = FOGTransformer()
print(f'The model has {count_parameters(model):,} trainable parameters')
model = FOGModel()
print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 1,674,243 trainable parameters
The model has 1,508,611 trainable parameters


In [162]:
tdcsfog_train_loader = DataLoader(tdcsfog_train, batch_size=cfg.batch_size, shuffle=True)

model = FOGModel()
optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr)
criterion = nn.BCEWithLogitsLoss()
soft = nn.Softmax(dim=-1)

# def average_precision_score(y_true, y_pred):
#         # average precision with pytorch
#         y = y_true.argmax(dim=-1)
#         average_precision = AveragePrecision(task="multiclass", num_classes=3, average=None)
#         return average_precision(y_pred, y)

def train(model, optimizer, criterion, train_loader):
    for x, y in tqdm(train_loader):
        # print(y)
        # print(x.shape, y.shape)
        #ic(x, y)
        # single forward pass
        # cast x to the correct data type
        x = x.float()
        y_hat = model(x)
        # print(y_hat)
        # print(soft(y_hat))
        # print(y_hat.shape)
        # print(y_hat.argmax(dim=-1))
        # calculate loss
        loss = criterion(y_hat, y)
        acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean()
        # calculate gradients
        loss.backward()
        # update weights
        optimizer.step()
        # print out the loss using ic
        print(loss.item())
        print(acc.item())
        print(average_precision_score(y, y_hat))
        break

def validation(model, criterion, valid_loader):
    for x, y in tqdm(valid_loader):
        # single forward pass
        # cast x to the correct data type
        x = x.float()
        # disable gradient calculation
        with torch.no_grad():
            y_hat = model(x)
        print(y_hat)
        print(y_hat.argmax(dim=-1))
        print(y)
        print(y.argmax(dim=-1))
        # calculate loss
        loss = criterion(y_hat, y)
        acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean()
        # print out the loss using ic
        print(loss.item())
        print(acc.item())
        print(average_precision_score(y, y_hat))
        break

print("Training")
train(model, optimizer, criterion, tdcsfog_train_loader)
print("Validation")
validation(model, criterion, tdcsfog_valid_loader)

Training


  0%|          | 0/5825 [00:00<?, ?it/s]

0.6507755896673189
0.517578125
tensor([0.7866, 0.1749, 0.0437])
Validation


  0%|          | 0/1073 [00:00<?, ?it/s]

tensor([[-0.2091,  0.1024, -0.1447],
        [-0.3653,  0.4241,  0.1794],
        [-1.0421,  0.2758,  0.2277],
        ...,
        [ 0.1595, -0.5969, -0.7561],
        [ 0.3503, -0.3253, -0.1526],
        [ 0.2390, -0.2021,  0.0899]])
tensor([1, 1, 1,  ..., 0, 0, 0])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)
tensor([0, 0, 0,  ..., 0, 0, 0])
0.5960417736981375
0.6435546875
tensor([1., nan, nan])


In [132]:
class FOGModule(pl.LightningModule):

    def __init__(self, model, optimizer_name, optimizer_hparams):
        """
        Inputs:
            model_name - Name of the model to run. Used for creating the model (see function below)
            model_hparams - Hyperparameters for the model, as dictionary.
            optimizer_name - Name of the optimizer to use. Currently supported: Adam, SGD
            optimizer_hparams - Hyperparameters for the optimizer, as dictionary. This includes learning rate, weight decay, etc.
        """
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = model
        # Create loss module
        self.loss_module = nn.BCEWithLogitsLoss()
        # Example input for visualizing the graph in Tensorboard
        self.example_input_array = torch.zeros((1, cfg.window_size, 3), dtype=torch.float32)

    def forward(self, past):
        # Forward function that is run when visualizing the graph
        return self.model(past)

    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            # AdamW is Adam with a correct implementation of weight decay (see here for details: https://arxiv.org/pdf/1711.05101.pdf)
            optimizer = torch.optim.AdamW(
                self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = torch.optim.SGD(self.parameters(), **self.hparams.optimizer_hparams)
        else:
            assert False, f"Unknown optimizer: \"{self.hparams.optimizer_name}\""

        # We will reduce the learning rate by 0.1 after 100 and 150 epochs
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[100, 150], gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        # "batch" is the output of the training data loader.
        past, future = batch
        past = past.float()
        future = future.float()
        preds = self.model(past)
        loss = self.loss_module(preds, future)
        acc = (preds.argmax(dim=-1) == future.argmax(dim=-1)).float().mean()
        self.log('train_acc', acc)

        ap = self.average_precision_score(future, preds)
        self.log('train_ap0', ap[0])
        self.log('train_ap1', ap[1])
        self.log('train_ap2', ap[2])
        self.log('train_ap', sum(ap)/3)

        # Logs the accuracy per epoch to tensorboard (weighted average over batches)
        #self.log('train_acc', acc, on_step=False, on_epoch=True)
        self.log('train_loss', loss)
        return loss  # Return tensor to call ".backward" on

    def validation_step(self, batch, batch_idx):
        past, future = batch
        past = past.float()
        future = future.float()
        preds = self.model(past)
        ap = self.average_precision_score(future, preds)
        acc = (future.argmax(dim=-1) == preds.argmax(dim=-1)).float().mean()
        self.log('val_acc', acc)

        # By default logs it per epoch (weighted average over batches)
        self.log('val_ap0', ap[0])
        self.log('val_ap1', ap[1])
        self.log('val_ap2', ap[2])
        self.log('val_ap', sum(ap)/3)

    def test_step(self, batch, batch_idx):
        past, future = batch
        past = past.float()
        future = future.float()
        preds = self.model(past)
        acc = (future.argmax(dim=-1) == preds.argmax(dim=-1)).float().mean()
        # By default logs it per epoch (weighted average over batches), and returns it afterwards
        self.log('test_acc', acc)
    
    def average_precision_score(self, y_true, y_pred):
        # average precision with pytorch
        y = y_true.argmax(dim=-1)
        average_precision = AveragePrecision(task="multiclass", num_classes=3, average=None)
        return average_precision(y_pred, y)
        

In [133]:
def train_model(module, model, train_loader, val_loader, test_loader, save_name = None, **kwargs):
    """
    Inputs:
        model_name - Name of the model you want to run. Is used to look up the class in "model_dict"
        save_name (optional) - If specified, this name will be used for creating the checkpoint and logging directory.
    """
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=os.path.join(cfg.CHECKPOINT_PATH, save_name),                          # Where to save models
                         accelerator="gpu" if str(cfg.device).startswith("cuda") else "cpu",                     # We run on a GPU (if possible)
                         devices=1,                                                                          # How many GPUs/CPUs we want to use (1 is enough for the notebooks)
                         max_epochs=2,                                                                     # How many epochs to train for if no patience is set
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_ap"),  # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
                                    LearningRateMonitor("epoch")],                                           # Log learning rate every epoch
                         enable_progress_bar=True,                                                          # Set to False if you do not want a progress bar
                         logger = True)                                                           
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = True # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(cfg.CHECKPOINT_PATH, save_name + ".ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = module.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        pl.seed_everything(42) # To be reproducable
        lmodel = module(model, **kwargs)
        trainer.fit(lmodel, train_loader, val_loader)
        lmodel = module.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(lmodel, val_loader, verbose=False)
    #test_result = trainer.test(model, test_loader, verbose=False)
    result = {"val": val_result[0]["test_acc"]}

    return lmodel, trainer, result

In [134]:
model = FOGModel()
model, trainer, result = train_model(FOGModule, model, tdcsfog_train_loader, tdcsfog_valid_loader, tdcsfog_test_loader, save_name="FOGModel", optimizer_name="Adam", optimizer_hparams={"lr": 0.001, "weight_decay": 0.0001})
result

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
/Users/taa/anaconda3/envs/parkinson/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(

  | Name        | Type              | Params | In sizes     | Out sizes
-----------------------------------------------------------------------------
0 | model       | FOGModel          | 1.5 M  | [1, 3840, 3] | [1, 3]   
1 | loss_module | BCEWithLogitsLoss | 0      | ?            | ?        
-----------------------------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.034     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/taa/anaconda3/envs/parkinson/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/taa/anaconda3/envs/parkinson/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
/Users/taa/anaconda3/envs/parkinson/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

{'val': 0.3162151277065277}

## Pre - Training

## Fine - Tuning

## Submission

In [135]:
model = FOGModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
model.to(cfg.device)
#model.eval()

test_defog_paths = glob.glob(f"{cfg.DATA_DIR}test/defog/*.csv")
test_tdcsfog_paths = glob.glob(f"{cfg.DATA_DIR}test/tdcsfog/*.csv")
#print(test_tdcsfog_paths)
test_fpaths = test_defog_paths + test_tdcsfog_paths

test_dataset = FOGDataset(test_fpaths, test=True)
test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size)

ids = []
preds = []

for _id, x in tqdm(test_loader):
    x = x.to(cfg.device).float()
    with torch.no_grad():
        y_pred = model(x)*0.1

    ids.extend(_id)
    preds.extend(list(np.nan_to_num(y_pred.cpu().numpy())))

sample_submission = pd.read_csv(f"{cfg.DATA_DIR}sample_submission.csv")
print(sample_submission.shape)

preds = np.array(preds)
submission = pd.DataFrame({'Id': ids, 'StartHesitation': np.round(preds[:,0],5), \
                           'Turn': np.round(preds[:,1],5), 'Walking': np.round(preds[:,2],5)})

submission = pd.merge(sample_submission[['Id']], submission, how='left', on='Id').fillna(0.0)
submission.to_csv(f"submission.csv", index=False)

print(submission.shape)
submission.head()

Dataset initialized in 0.1937119960784912 secs!


  0%|          | 0/280 [00:00<?, ?it/s]

(286370, 4)
(286370, 4)


,Id,StartHesitation,Turn,Walking
0,003f117e14_0,-0.32976,-0.12395,-0.41426
1,003f117e14_1,-0.31996,-0.10720,-0.34863
2,003f117e14_2,-0.27493,-0.09433,-0.45884
3,003f117e14_3,-0.34387,-0.10961,-0.43158
4,003f117e14_4,-0.27932,-0.11053,-0.38124
